In [105]:
import torch
import numpy as np
import torch.nn as nn
from torchvision.datasets import MNIST
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from IPython.display import clear_output as co
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.optim import Adam
import seaborn as sns
from PIL import Image
from glob import glob
sns.set_theme(style="darkgrid", font_scale = 0.7)

try:
    from dotenv import load_dotenv
    load_dotenv()
except:
    print('.env is not loaded')
    pass

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from teleflow import Bot
bot = Bot()

In [106]:
def _load(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('L')

class Faces(Dataset):
    def __init__(self, root, split = 'train', transform = None, download = False):
        if download:
            self._download()
        
        self.root = os.path.join(root, split)
        self.names, self.name2id, self.id2name = self._get_names()
        self.images, self.labels = self._get_people()
        self.total_idxs = np.arange(len(self.images))
        self.transform = transform



    def _get_names(self):
        names = list(map(lambda x : int(x), os.listdir(self.root)))
        name2id = {name : _ for _, name in enumerate(names)}
        id2name = {_ : name for _, name in enumerate(names)}
 
        return names, name2id, id2name
    
    def _get_people(self):
        images = glob(os.path.join(self.root, '*\\*'))
        labels = [self.name2id[int(image.split('\\')[-2])] for image in images]
        return np.array(images), np.array(labels)
    
    def _download(self):
        import gdown
        import zipfile
        import os
        import shutil
        if os.path.exists('faces_dataset'):
            shutil.rmtree('faces_dataset')
        file_id = '1S6Wqm9CoLgVVWe8kPIbBseS_ndGTIR-V'
        output = 'faces_dataset.zip'
        gdown.download(f'https://drive.google.com/uc?id={file_id}', output)
        with zipfile.ZipFile(output, 'r') as zip_ref:
            zip_ref.extractall('faces_dataset_unzipped')
            shutil.move('./faces_dataset_unzipped/faces_dataset', './faces_dataset')
        os.remove(output)
        os.rmdir('./faces_dataset_unzipped')
        print('Загрузка датасета произошла успешно бип-буп-буп')
            

    def __getitem__(self, index):
        image1 = _load(self.images[index])

        if np.random.uniform() < 0.5:
            image2 = _load(np.random.choice(self.images[self.labels == self.labels[index]]))
            label = torch.tensor([1], dtype = torch.float)
        else:
            image2 = _load(np.random.choice(self.images[self.labels != self.labels[index]]))
            label = torch.tensor([0], dtype = torch.float)
        
        if self.transform is not None:
            image1 = self.transform(image1)
            image2 = self.transform(image2)
        
        return image1, image2, label, torch.tensor(self.labels[index], dtype = torch.int)
    
    def __len__(self):
        return len(self.images)

faces = Faces(root = 'faces_dataset', download = True)

Downloading...
From: https://drive.google.com/uc?id=1S6Wqm9CoLgVVWe8kPIbBseS_ndGTIR-V
To: c:\Users\rockxi\Desktop\учеба\python\homework\faces_dataset.zip
100%|██████████| 2.83M/2.83M [00:00<00:00, 19.2MB/s]


Загрузка датасета произошла успешно бип-буп-буп


In [107]:
train_transform = T.Compose([
    T.Resize((128, 128)),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
])
test_trainform = T.Compose([
    T.Resize((128, 128)),
    T.ToTensor(),
])

train_set = Faces(root = 'faces_dataset', split = 'train', transform = train_transform)
test_set = Faces(root = 'faces_dataset', split = 'test', transform = test_trainform)

train_loader = DataLoader(train_set, batch_size = 30 , shuffle = True, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_set, batch_size = 1, shuffle = False, num_workers=0, pin_memory=True)

In [108]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin = 2.0):
        super().__init__()
        self.margin = margin
    def forward(self, output1, output2, label):
        euclidean_distance = nn.functional.pairwise_distance(output1, output2, keepdim=True)
        return torch.mean(label * torch.pow(euclidean_distance, 2) + (1 - label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min = 0.0), 2))

class ContrativeModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.feature_ex = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1, 1),
            nn.BatchNorm2d(32),
            nn.ELU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 32, 3, 1, 1),
            nn.BatchNorm2d(32),
            nn.ELU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 32, 3, 1, 1),
            nn.BatchNorm2d(32),
            nn.ELU(),
            nn.MaxPool2d(2),

            nn.Flatten(),
        )
        self.fc = nn.Sequential(
            nn.Linear(16 * 16 * 32, 512),
            nn.ELU(),
            nn.Linear(512, 10),
        )

    def forward_single(self, x):
        return self.fc(self.feature_ex(x))
        
    def forward(self, x1, x2):
        return self.forward_single(x1), self.forward_single(x2)

In [109]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'; print(device)

cuda


In [110]:
def train(model, loader, optim, loss_fn):
    model.train()
    train_loss = 0
    for x1, x2, y, _ in tqdm(loader, desc = "Train"):
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        optim.zero_grad()
        y_pred1, y_pred2 = model(x1, x2)
        loss = loss_fn(y_pred1, y_pred2, y)
        train_loss += loss.item()
        loss.backward()
        optim.step()
    return train_loss / len(loader)
@torch.inference_mode()
def get_latent_base(model):
    model.eval()
    xs = []
    ys = []
    zs = []

    for x, _, _, y in train_loader:
        z = model.forward_single(x.to(device))
        xs.append(x)
        ys.append(y)
        zs.append(z)
    xs = torch.cat(xs)
    ys = torch.cat(ys)
    zs = torch.cat(zs)
    return xs, ys, zs
@torch.inference_mode()
def test(model, loader):
    model.eval()
    xs, ys, zs = get_latent_base(model)
    
    total, correct = 0, 0
    for i, (x, _, _, y) in tqdm(enumerate(loader), desc = 'Test'):
        z = model.forward_single(x.to(device))
        dist = nn.functional.pairwise_distance(z, zs)
        idx = torch.argmin(dist) 
        correct += ys.to(device)[idx] == y.to(device)
        total += 1
    return (correct / total).cpu()

def octava(model, train_loader, test_loader, optim, loss_fn, epochs = 10, model_name = 'default'):
    train_loss_arr, test_acc_arr = [], []
    for epoch in range(epochs):
        plt.figure(figsize = (10, 5))
        train_loss = train(model, train_loader, optim, loss_fn)
        test_acc = test(model, test_loader)
        train_loss_arr.append(train_loss)
        test_acc_arr.append(test_acc)
        lr_shed.step(train_loss)
        co(wait = True)
        if epoch == epochs - 1:
            plt.plot(train_loss_arr, label = 'Train Loss')
            plt.plot(test_acc_arr, label = 'Test Acc')
            plt.legend()
            plt.savefig(model_name + '.png')
        else: 
            plt.plot(train_loss_arr, label = 'Train Loss')
            plt.plot(test_acc_arr, label = 'Test Acc')
            plt.legend()
        plt.show()
        print(f'Epoch: {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Test Acc: {float(test_acc):.4f}')


In [103]:
model = ContrativeModel().to(device)

In [104]:
optim = Adam(model.parameters(), lr = 1e-3)
loss_fn = ContrastiveLoss()
lr_shed = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, factor=0.5, patience=5)
model_name = 'stats'
epochs = 60
if model_name not in os.listdir():
    octava(model, train_loader, test_loader, optim, loss_fn, epochs, model_name)
    torch.save(model.state_dict(), model_name + '.pth')
    bot.send_results(text = f'Обучение {model_name} на {epochs} эпохах завершено.', image = model_name + '.png', file = model_name + '.pth')
else:
    model.load_state_dict(torch.load(model_name + '.pth', weight_only = True))

Train:  67%|██████▋   | 2/3 [00:09<00:04,  4.53s/it]


KeyboardInterrupt: 

<Figure size 1000x500 with 0 Axes>